In [28]:
import pandas as pd
import os
import re

# === CONFIG ===
files = [
    r"...lumniPhase1\SAS outputs\Analyst Tables\PH Ed Coding\Alumni_PH Ed Coding_EM.xlsx",
    r"...AlumniPhase1\SAS outputs\Analyst Tables\PH Ed Coding\Alumni_PH Ed Coding_NK.xlsx",
]
filestesting = [

    r"...\Agreement Testing_Empl.xlsx",
]
STAGE = '@PROJECT_DB.DATASETS.PROJECT_DB_Stage'
FILE_FORMAT = 'CSV_FORMAT'
DATABASE = 'PROJECT_DB'
SCHEMA = 'DATASETS'

# === Helpers ===
def clean_name(s):
    s = re.sub(r"[^\w]+", "_", s)       # replace non-alphanum with underscore
    return re.sub(r'_+', '_', s).strip('_').upper()

def clean_filename(file_path, sheet=None):
    base = os.path.splitext(os.path.basename(file_path))[0]
    if sheet: base += f"_{sheet}"
    return clean_name(base) + '.csv'

def process_df(df, path):
    df.columns = [clean_name(c) for c in df.columns]
    #df.to_csv(path, index=False)
    return df

# === Process files ===
cleaned = []

for f in files:
    ext = os.path.splitext(f)[1].lower()
    if ext in ['.csv', '.tsv']:
        df = pd.read_csv(f, sep=',' if ext=='.csv' else '\t', low_memory=False)
        out = os.path.splitext(f)[0] + "_cleaned.csv"
        process_df(df, out)
        cleaned.append((out, clean_filename(out)))
    elif ext == '.xlsx':
        xls = pd.ExcelFile(f)
        for sh in xls.sheet_names:
            df = pd.read_excel(xls, sheet_name=sh)
            sh = clean_name(sh)
            out = os.path.splitext(f)[0] + f"_{sh}_cleaned.csv"
            process_df(df, out)
            cleaned.append((out, clean_filename(f, sh)))

# === Snowflake PUT statements ===
print("\n-- PUT statements --")
for local, stage_file in cleaned:
    path = os.path.abspath(local).replace("\\","/")
    print(f'snow sql -q "PUT \'file://{path}\' {STAGE}/{stage_file} AUTO_COMPRESS=FALSE OVERWRITE=TRUE;"')

# === CREATE TABLE + COPY INTO ===
print("\n-- CREATE TABLE & COPY INTO --")

for local, tbl_full in cleaned:
    df = pd.read_csv(local, nrows=0)
    cols = ",\n    ".join([f'"{c}" VARCHAR' for c in df.columns])
    tbl = tbl_full.removesuffix(".csv")
    create = f"CREATE OR REPLACE TABLE {DATABASE}.{SCHEMA}.{tbl} (\n    {cols}\n);"
    copy = f"""
COPY INTO {DATABASE}.{SCHEMA}.{tbl}
FROM {STAGE}/{tbl_full}
FILE_FORMAT = (FORMAT_NAME = {FILE_FORMAT})
ON_ERROR = 'CONTINUE';
""".strip()
    print(f"\n-- {tbl} --\n{create}\n{copy}\n")



-- PUT statements --
snow sql -q "PUT 'file://V:/Aguirre Projects/CNCS Public Health AmeriCorps Evaluations/Task 5. Data Analysis/Outcome Evaluation/AlumniPhase1/SAS outputs/Analyst Tables/PH Ed Coding/Alumni_PH Ed Coding_EM_CURRENT_ENROLLMENT_CODING_cleaned.csv' @PROJECT_DB.DATASETS.PROJECT_DB_Stage/ALUMNI_PH_ED_CODING_EM_CURRENT_ENROLLMENT_CODING.csv AUTO_COMPRESS=FALSE OVERWRITE=TRUE;"
snow sql -q "PUT 'file://V:/Aguirre Projects/CNCS Public Health AmeriCorps Evaluations/Task 5. Data Analysis/Outcome Evaluation/AlumniPhase1/SAS outputs/Analyst Tables/PH Ed Coding/Alumni_PH Ed Coding_EM_DATA_DICTIONARY_cleaned.csv' @PROJECT_DB.DATASETS.PROJECT_DB_Stage/ALUMNI_PH_ED_CODING_EM_DATA_DICTIONARY.csv AUTO_COMPRESS=FALSE OVERWRITE=TRUE;"
snow sql -q "PUT 'file://V:/Aguirre Projects/CNCS Public Health AmeriCorps Evaluations/Task 5. Data Analysis/Outcome Evaluation/AlumniPhase1/SAS outputs/Analyst Tables/PH Ed Coding/Alumni_PH Ed Coding_NK_CURRENT_ENROLLMENT_CODING_cleaned.csv' @PROJECT_DB.D